In [3]:
# epistemoverse_s_test.py
import re
import numpy as np
import ollama
import plotly.graph_objects as go

# --- cosine similarity ---
def cos(a, b):
    a, b = np.array(a, dtype=float), np.array(b, dtype=float)
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-8))

# --- parse dialogue from text file ---
def parse_dialogue_multiline(file_path):
    dialogue = []
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    lines = lines[2:]  # skip first line (question) and second line

    i = 0
    while i < len(lines):
        line = lines[i].strip()
        if line.startswith("Index:"):
            turn = {}
            turn['Index'] = int(re.match(r"Index:\s*(\d+)", line).group(1))

            i += 1
            turn['Speaker'] = re.match(r"Speaker:\s*(.+)", lines[i].strip()).group(1)

            i += 1
            turn['Keys'] = re.match(r"Keys:\s*(.*)", lines[i].strip()).group(1)

            i += 1
            # Handle multi-line Text until next line starting with "Question:"
            text_lines = []
            while i < len(lines) and not lines[i].startswith("Question:"):
                text_lines.append(lines[i].strip())
                i += 1
            turn['Text'] = " ".join(text_lines)

            # Question (optional)
            if i < len(lines) and lines[i].startswith("Question:"):
                turn['Question'] = re.match(r"Question:\s*(.*)", lines[i].strip()).group(1)
                i += 1
            else:
                turn['Question'] = ""

            dialogue.append(turn)
        else:
            i += 1
    return dialogue

# --- compute awareness & integration series ---
def awareness_integration_series(steps, awareness_anchor, integration_anchor, embed_model="nomic-embed-text"):
    awareness_scores = []
    integration_scores = []
    for s in steps:
        if not s.strip():  # skip empty text
            continue
        e = ollama.embeddings(model=embed_model, prompt=s)["embedding"]
        awareness_scores.append(cos(e, awareness_anchor))
        integration_scores.append(cos(e, integration_anchor))
    return awareness_scores, integration_scores

# --- interactive 3D plot ---
def plot_3d_interactive(steps, series, speakers, label="Epistemoverse Dialogue"):
    fig = go.Figure()
    x = list(range(1, len(steps)+1))
    y, z = series
    
    # assign colors per speaker
    speaker_colors = {}
    color_palette = ["blue", "red", "green", "orange", "purple", "brown"]
    for idx, sp in enumerate(sorted(set(speakers))):
        speaker_colors[sp] = color_palette[idx % len(color_palette)]
    colors = [speaker_colors[s] for s in speakers]

    fig.add_trace(go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers+lines',
        marker=dict(size=5, color=colors),
        line=dict(color='black', width=2),
        text=[f"{s}: {t}" for s,t in zip(speakers, steps)],
        hovertemplate='Step %{x}:<br>Awareness: %{y:.3f}<br>Integration: %{z:.3f}<br>%{text}',
        name=label
    ))

    fig.update_layout(
        title="Interactive Epistemoverse S-test 3D",
        scene=dict(
            xaxis_title='Step in Dialogue',
            yaxis_title='Awareness (cosine)',
            zaxis_title='Integration (cosine)'
        ),
        width=900,
        height=600
    )
    fig.show()

# --- main function ---
if __name__ == "__main__":
    file_path = "Complete.txt"  # replace with your file

    # parse dialogue
    dialogue = parse_dialogue(file_path)
    steps = [turn['Text'] for turn in dialogue]
    speakers = [turn['Speaker'] for turn in dialogue]

    # define anchors
    awareness_anchor = ollama.embeddings(model="nomic-embed-text", prompt="awareness")["embedding"]
    integration_anchor = ollama.embeddings(model="nomic-embed-text", prompt="integration")["embedding"]

    # compute series
    series = awareness_integration_series(steps, awareness_anchor, integration_anchor)

    # plot interactive 3D
    plot_3d_interactive(steps, series, speakers)


In [6]:
# epistemoverse_tsne_plot.py
import re
import numpy as np
import plotly.express as px
from sklearn.manifold import TSNE
import ollama

# ----------------------------
# --- Utility functions ---
# ----------------------------

def cos(a,b):
    a,b = np.array(a), np.array(b)
    return float(np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b)+1e-8))

def prepare_hover_text(speakers, steps, char_limit=300, line_width=80):
    hover_texts = []
    for sp, t in zip(speakers, steps):
        t_trunc = t[:char_limit] + "..." if len(t) > char_limit else t
        t_lines = '<br>'.join([t_trunc[i:i+line_width] for i in range(0, len(t_trunc), line_width)])
        hover_texts.append(f"{sp}<br>{t_lines}")
    return hover_texts

# ----------------------------
# --- Parse dialogue ---
# ----------------------------

def parse_dialogue_multiline(file_path):
    dialogue = []
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    # skip first two lines
    lines = lines[2:]

    i = 0
    while i < len(lines):
        line = lines[i].strip()
        if line.startswith("Index:"):
            turn = {}
            turn['Index'] = int(re.match(r"Index:\s*(\d+)", line).group(1))

            i += 1
            turn['Speaker'] = re.match(r"Speaker:\s*(.+)", lines[i].strip()).group(1)

            i += 1
            turn['Keys'] = re.match(r"Keys:\s*(.*)", lines[i].strip()).group(1)

            i += 1
            # Collect multi-line Text until "Question:" or end
            text_lines = []
            while i < len(lines) and not lines[i].startswith("Question:") and not lines[i].startswith("Index:"):
                text_lines.append(lines[i].strip())
                i += 1
            turn['Text'] = " ".join(text_lines)

            # Question (optional)
            if i < len(lines) and lines[i].startswith("Question:"):
                turn['Question'] = re.match(r"Question:\s*(.*)", lines[i].strip()).group(1)
                i += 1
            else:
                turn['Question'] = ""

            dialogue.append(turn)
        else:
            i += 1
    return dialogue

# ----------------------------
# --- Awareness / Integration ---
# ----------------------------

def awareness_integration_series(steps, awareness_anchor, integration_anchor, embed_model="nomic-embed-text"):
    awareness_scores = []
    integration_scores = []
    embeddings = []
    for s in steps:
        if not s.strip():
            continue
        e = ollama.embeddings(model=embed_model, prompt=s)["embedding"]
        embeddings.append(e)
        awareness_scores.append(cos(e, awareness_anchor))
        integration_scores.append(cos(e, integration_anchor))
    return np.array(embeddings), np.array(awareness_scores), np.array(integration_scores)

# ----------------------------
# --- Main ---
# ----------------------------

if __name__ == "__main__":
    file_path = "Complete.txt"  # replace with your file

    # parse dialogue
    dialogue = parse_dialogue_multiline(file_path)
    steps = [turn['Text'] for turn in dialogue if turn['Text'].strip()]
    speakers = [turn['Speaker'] for turn in dialogue if turn['Text'].strip()]

    # anchors
    awareness_anchor = ollama.embeddings(model="nomic-embed-text", prompt="awareness")["embedding"]
    integration_anchor = ollama.embeddings(model="nomic-embed-text", prompt="integration")["embedding"]

    # embeddings + awareness/integration
    embeddings, awareness_scores, integration_scores = awareness_integration_series(
        steps, awareness_anchor, integration_anchor
    )

    # --- dimensionality reduction ---
    tsne = TSNE(n_components=2, random_state=42, perplexity=5)
    emb_2d = tsne.fit_transform(embeddings)

    # --- prepare hover texts ---
    hover_texts = prepare_hover_text(speakers, steps)

    # --- interactive plot ---
    fig = px.scatter(
        x=emb_2d[:,0],
        y=emb_2d[:,1],
        color=awareness_scores,
        size=integration_scores,
        hover_name=speakers,
        hover_data={"Text": hover_texts, "Awareness": awareness_scores, "Integration": integration_scores},
        title="Epistemoverse Dialogue Clusters (t-SNE)",
        color_continuous_scale="Viridis"
    )

    fig.update_layout(width=900, height=600)
    fig.show()
